In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt